In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

In [3]:
path = "dog_breed/"
train_path = os.path.join(path, "train/*")
test_path = os.path.join(path, "test/*")
labels_path = os.path.join(path, "labels.csv")


In [4]:
labels_df = pd.read_csv(labels_path)
breed = labels_df["breed"].unique()
print("Number of Breed: ", len(breed))

Number of Breed:  120


In [5]:
breed2id = {name: i for i, name in enumerate(breed)}
id2breed = {i: name for i, name in enumerate(breed)}

In [6]:
ids = glob(train_path)
labels = []

In [7]:
for image_id in ids:
    image_id = image_id.split("/")[-1].split("n")[-1].split(".")[0]
    try:
        breed_name = list(labels_df[labels_df.id == image_id[1:]]["breed"])[0]
        breed_idx = breed2id[breed_name]
        labels.append(breed_idx)
    except:
        print(image_id[:])

In [8]:
ids = ids[:1000]
labels = labels[:1000]

In [9]:
## Spliting the dataset
train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42)
train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)

In [11]:
 ## Model
model = tf.keras.models.load_model("model.h5")
for i, path in tqdm(enumerate(valid_x[:10])):
    image = read_image(path, 224)
    image = np.expand_dims(image, axis=0)
    pred = model.predict(image)[0]
    label_idx = np.argmax(pred)
    breed_name = id2breed[label_idx]

    ori_breed = id2breed[valid_y[i]]
    ori_image = cv2.imread(path, cv2.IMREAD_COLOR)

    ori_image = cv2.putText(ori_image, breed_name, (0, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    ori_image = cv2.putText(ori_image, ori_breed, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    cv2.imwrite(f"save/valid_{i}.png", ori_image)

10it [00:01,  7.44it/s]
